In [60]:
import os
import fitz
from tqdm import tqdm

In [61]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [62]:
possible_rate

['BUY', 'OUTPERFORM', 'NEUTRAL', 'UNDERPERFORM', 'SELL', None]

In [63]:
zip_path = r'..\file\CLSA'
count_true = 0
possible_rate = ['BUY', 'OUTPERFORM', 'NEUTRAL', 'UNDERPERFORM', 'SELL']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if 'CL Securities' in text_check_source :
                # 提取評價的第一種方法
                try:
                    clip_old_report_1 = fitz.Rect(160, 0, rect.width, 100)
                    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
                    text_old_report_1 = text_old_report_1.split('-')[1].strip()
                    rate_1 = text_old_report_1.split('\n')[0].strip()
                except:
                    rate_1 = None
                    # 提取評價的第二種方法
                try:
                    page_old_report_2 = doc.load_page(1)
                    clip_old_report_2 = fitz.Rect(0, 0, rect.width, 60)
                    text_old_report_2 = page_old_report_2.get_text(clip=clip_old_report_2, sort=True).strip()
                    if 'O-PF' in text_old_report_2:
                        rate_2 = 'OUTPERFORM'
                    elif 'U-PF' in text_old_report_2:
                        rate_2 = 'UNDERPERFORM'
                    else:
                        rate_2 = text_old_report_2.split('-')[1].strip()
                except:
                    rate_2 = None
                # if rate_1 == rate_2 and rate_1 not in possible_rate:
                #     possible_rate.append(rate_1)
                rate = check_rate_function(rate_1, rate_2, possible_rate)
            else:
                # False
                print(f'{directory_path} is not posted by hsbc')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print('case1', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print('case2', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

100%|██████████| 13/13 [00:00<00:00, 15.95it/s]

case1 ..\file\CLSA\5347-世界-2018_06_07-NULL-NULL.pdf NULL rate_1=None, rate_2=None
case1 ..\file\CLSA\6488-環球晶-2018_06_07-NULL-NULL.pdf NULL rate_1=None, rate_2=None
共13件, 成功11件, 失敗2件, 成功率:84.6%


In [64]:
import fitz
doc = fitz.open(r'..\file\CLSA\3034-聯詠-2023_02_08-cl-OUTPERFORM.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [65]:
# 檢查是否由第一金投顧出版
page_check_source = doc.load_page(-1)
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-clip_check_source.png")
text_check_source = page_check_source.get_text()
if 'CL Securities' in text_check_source :
    print('True')
text_check_source

True


'Important disclosures\nNovatek - O-PF\n7 February 2023\njason.tsang@cl-sec.com\n15\nCLSA Europe BV, authorised and regulated by the Netherlands \nAuthority for Financial Markets. \nFor all other jurisdiction-specific disclaimers please refer to \nhttps://www.clsa.com/disclaimer.html. The analysts/contributors to \nthis report may be employed by any relevant CLSA entity or CLST, \nwhich is different from the entity that distributes the report in the \nrespective jurisdictions.© 2023 CLSA and/or CL Securities Taiwan \nCo., Ltd. (“CLST”).\n'

In [66]:
# 檢查版本
clip_check_report = fitz.Rect(350, 0, rect.width, 250)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_report)
pix.save("page-clip_check_report.png")
text_check_report = page.get_text(clip=clip_check_report)
if 'Equities' in text_check_report :
    print('Equities')
else :
    print('NULL')
text_check_report

NULL


'l\nsus with a recovery in LDDIC and OLED driver \nto be flat to slightly up in 1Q23 with GPM of \nhich are above market forecasts. It believes\n 1Q23 and stabilise from 2Q23, and expects\nny is positive on OLED driver, high-end IT and \n revise up 23/24CT EPS by 23/23% and lift \n'

In [67]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(160, 0, rect.width, 100)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_new_report_1.png")
    text_old_report_1 = text_old_report_1.split('-')[1].strip()
    rate_1 = text_old_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

'OUTPERFORM'

In [68]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    page_old_report_2 = doc.load_page(1)
    clip_old_report_2 = fitz.Rect(0, 0, rect.width, 60)
    text_old_report_2 = page_old_report_2.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_new_report_2.png")
    # text_old_report_2 = text_old_report_2.split('-')[1].strip()
    if 'O-PF' in text_old_report_2:
        rate_2 = 'OUTPERFORM'
    elif 'U-PF' in text_old_report_2:
        rate_2 = 'UNDERPERFORM'
    else:
        rate_2 = text_old_report_2.split('-')[1].strip()
except:
    rate_2 = None
rate_2

'OUTPERFORM'